In [1]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments
)
from optimum.intel.neural_compressor import INCTrainer
from datasets import load_dataset
import evaluate
import numpy as np

# --- 配置 ---
MODEL_NAME = "bert-base-uncased"
TOKENIZER_NAME = "bert-base-uncased" 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("所有库已导入，配置完成！")

2025-09-16 02:00:29 [WARNING][auto_accelerator.py:454] Auto detect accelerator: CUDA_Accelerator.
2025-09-16 02:00:29 [WARNING][auto_accelerator.py:454] Auto detect accelerator: CUDA_Accelerator.
2025-09-16 02:00:29 [WARNING][fp_utils.py:127] hw aligned scales not supported for device INCAcceleratorType.CUDA


所有库已导入，配置完成！


In [2]:
# 加载和预处理数据集
raw_datasets = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

# 加载【预训练】模型
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# 定义评估函数
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print("数据集和模型加载完成！")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


数据集和模型加载完成！


In [3]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="./qat_results_optimum",
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_strategy="steps",
    logging_steps=20,
    eval_strategy="steps",
    save_strategy="no",
    eval_steps=200,
    load_best_model_at_end=False,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    report_to="wandb", # 你可以继续使用 wandb
)



# 使用 INCTrainer
trainer = INCTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("INCTrainer 已创建，即将开始量化感知训练...")

# 启动训练
trainer.train()

print("量化感知训练完成！")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


INCTrainer 已创建，即将开始量化感知训练...


wandb: Currently logged in as: maniaamaeovo (maniaamaeovo-mania) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
200,0.238000,0.244236,0.905963
400,0.194100,0.230937,0.909404
600,0.170900,0.214290,0.923165
800,0.200800,0.209781,0.918578
1000,0.160000,0.197130,0.930046
1200,0.104200,0.258058,0.908257
1400,0.091700,0.255760,0.910550
1600,0.095700,0.244576,0.913991
1800,0.095300,0.249569,0.910550
2000,0.104300,0.231465,0.925459


量化感知训练完成！


In [4]:
qat_model_save_path = "./saved_models/bert_sst2_int8_qat_optimum_batch64"

print(f"正在将最佳QAT模型保存到: {qat_model_save_path}")
trainer.save_model(qat_model_save_path)
print("QAT模型保存成功！")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


正在将最佳QAT模型保存到: ./saved_models/bert_sst2_int8_qat_optimum
QAT模型保存成功！
